In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
%matplotlib inline
from scipy import stats

In [2]:
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

In [3]:
from sklearn.linear_model import Ridge, Lasso, ElasticNet

In [4]:
from regression_function_fix import regx3

In [ ]:
# ridge = Ridge()
# lasso = Lasso()
# net   = ElasticNet()

In [5]:
model_vars_data = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\Data Science Bootcamp August\Machine Learning Project\data\model_vars_data.csv',index_col=0)

test_vars_data = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\Data Science Bootcamp August\Machine Learning Project\data\test_vars_data.csv',index_col=0)

price_train = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\Data Science Bootcamp August\Machine Learning Project\data\y_train.csv',index_col=0)

price_test = pd.read_csv(r'C:\Users\dgarb\OneDrive\Documents\Data Science Bootcamp August\Machine Learning Project\data\y_test.csv',index_col=0)

In [6]:
def miss_cols(df):
    missing = np.sum(df.isnull(),axis=0)
    
    miss_ind = np.sum(df.isnull(),axis=0) > 0
    
    vals = pd.Series(filter(lambda x: x != 0,missing))
     
    cols_miss = df.columns[miss_ind]

    L=list(zip(cols_miss,vals, round(vals/df.shape[0],4)))
    
    return sorted(L,reverse=True,key=lambda x: x[2])  

In [ ]:
miss_cols(model_vars_data)


In [7]:
model_vars_data.drop(['GarageYrBlt'],axis=1, inplace = True)
miss_cols(model_vars_data)

[]

In [8]:
test_vars_data.drop(['GarageYrBlt'],axis=1, inplace = True)
miss_cols(test_vars_data)

[]

In [9]:
price_train.head(2)

,SalePrice
1711,123000
1729,154400


In [10]:
pd.set_option('display.max_columns', None)
model_vars_data.head(3)

,bc_GrLivArea,log_LotArea,log_MasVnrArea,yeo_TotalBsmtSF,log_first_FlrSF,yeo_GarageArea,log_WoodDeckSF,log_OpenPorchSF,has_pool,log_BsmtFinSF1,log_2ndFlrSF,log_LotFrontage,log_BsmtUnfSF,log_EnclosedPorch,BsmtFinType1,YearRemodAdd,ExterCond,BsmtExposure,BsmtFinSF2,EnclosedPorch,BsmtHalfBath,Fireplaces,YearBuilt,BsmtFinType2,OverallQual,ScreenPorch,GarageQual,BsmtFullBath,GarageCond,BsmtFinSF1,HalfBath,LandSlope,FireplaceQu,LotFrontage,FullBath,BsmtQual,LotShape,HeatingQC,YrSold,KitchenQual,TotRmsAbvGrd,KitchenAbvGr,Functional,BedroomAbvGr,GarageCars,ExterQual,OverallCond,BsmtCond,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_None,Alley_Pave,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSewr,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_PreCast,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_PreCast,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mixed,Electrical_SBrkr,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_No,GarageFinish_No,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_P,PavedDrive_Y,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_No,MoSold_10,MoSold_11,MoSold_12,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
1711,8.306749,9.271812,0.0,197.803808,7.184629,115.298257,4.204693,0.000000,False,6.061457,0.000000,4.382027,0.000000,0.000000,4,1953,3,0,180.0,0,0.0,0,1953,2,5,0,3,1.0,3,428.0,0,3,0,79.0,1,3,4,3,2009,3,5,1,6,3,1.0,3,6,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,

In [11]:
pd.set_option('display.max_columns', None)
test_vars_data.head(3)

,bc_GrLivArea,log_LotArea,log_MasVnrArea,yeo_TotalBsmtSF,log_first_FlrSF,yeo_GarageArea,log_WoodDeckSF,log_OpenPorchSF,has_pool,log_BsmtFinSF1,log_2ndFlrSF,log_LotFrontage,log_BsmtUnfSF,log_EnclosedPorch,BsmtFinType1,YearRemodAdd,ExterCond,BsmtExposure,BsmtFinSF2,EnclosedPorch,BsmtHalfBath,Fireplaces,YearBuilt,BsmtFinType2,OverallQual,ScreenPorch,GarageQual,BsmtFullBath,GarageCond,BsmtFinSF1,HalfBath,LandSlope,FireplaceQu,LotFrontage,FullBath,BsmtQual,LotShape,HeatingQC,YrSold,KitchenQual,TotRmsAbvGrd,KitchenAbvGr,Functional,BedroomAbvGr,GarageCars,ExterQual,OverallCond,BsmtCond,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_C (all),MSZoning_FV,MSZoning_I (all),MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_None,Alley_Pave,LandContour_HLS,LandContour_Low,LandContour_Lvl,Utilities_NoSewr,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_Greens,Neighborhood_GrnHill,Neighborhood_IDOTRR,Neighborhood_Landmrk,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_PreCast,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Plywood,Exterior2nd_PreCast,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mixed,Electrical_SBrkr,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_No,GarageFinish_No,GarageFinish_RFn,GarageFinish_Unf,PavedDrive_P,PavedDrive_Y,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_No,MoSold_10,MoSold_11,MoSold_12,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
953,10.766258,9.017968,0.000000,345.800416,6.959399,247.305063,5.484797,0.0,False,5.666427,0.000000,4.110874,6.641182,0.000000,5,1963,3,0,0.0,0,0.0,0,1963,1,5,109,3,1.0,3,288.0,0,3,0,60.0,1,3,4,4,2007,4,6,1,8,3,2.0,3,7,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,1,0,

In [12]:
price_test.head(2)

,SalePrice
953,142100
330,135000


In [ ]:
model_vars_data.bc_GrLivArea

In [ ]:
test_vars_data.bc_GrLivArea

In [ ]:
def to_keep(it_lasso, amt):
    keeps1 = it_lasso.loc[it_lasso['abs_val'] > amt,['features']]
    list_keeps1 = list(keeps1.features)
    return list_keeps1

In [ ]:
it1_lasso = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data,
x_test = test_vars_data,
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keep1a =  list(filter(lambda x: x!= 'bc_GrLivArea',model_vars_data.columns))

In [ ]:
it_lasso1a = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keep1a],
x_test = test_vars_data[keep1a],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
it1_lasso.to_csv(r'C:\Users\dgarb\OneDrive\Documents\Data Science Bootcamp August\Machine Learning Project\linear model iterations\it1_lasso.csv')

In [ ]:
keeps = to_keep(it1_lasso,0.000053)
keeps

In [ ]:
it_lasso2 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso1a,0.000053)
keeps

In [ ]:
keeps = to_keep(it_lasso1a,0.000021)
# keeps

In [ ]:
it_lasso2a = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso2,0.000052)
keeps

In [ ]:
it_lasso3 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso1a, 0.001000)
# keeps

In [ ]:
it_lasso3a = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso1a, 0.002000)

In [ ]:
it_lasso3b = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso3,0.0007)
keeps

In [ ]:
it_lasso4 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso4,0.0008)
keeps

In [ ]:
it_lasso5 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso5,0.001)
keeps

In [ ]:
it_lasso6 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso6,0.002)
keeps

In [ ]:
it_lasso7 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso7,0.003)
type(keeps)

In [ ]:
it_lasso8 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso8,0.004)
type(keeps)

In [ ]:
it_lasso9 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso9,0.005)
type(keeps)

In [ ]:
it_lasso10 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso10,0.006)
type(keeps)

In [ ]:
it_lasso11 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso11,0.008)
type(keeps)

In [ ]:
it_lasso12 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso12,0.01)
keeps

In [ ]:
it_lasso13 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso13,0.015)
type(keeps)
keeps

In [ ]:
it_lasso14 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso14,0.017)
type(keeps)
keeps

In [ ]:
it_lasso15 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso15,0.019)
type(keeps)
keeps

In [ ]:
it_lasso16 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso16,0.03)
type(keeps)
keeps

In [ ]:
it_lasso17 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso17,0.03)
type(keeps)
keeps

In [ ]:
it_lasso18 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso18,0.04)
type(keeps)
keeps

In [ ]:
it_lasso19 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso19,0.044)
type(keeps)
keeps

In [ ]:
it_lasso20 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso20,0.06234)
type(keeps)
keeps

In [ ]:
it_lasso21 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_lasso21,0.064)
type(keeps)
keeps

In [ ]:
it_lasso22 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
# keeps = to_keep(it_lasso21,0.064)
# type(keeps)
keeps = ['OverallQual', 'bc_GrLivArea', 'YearBuilt']

In [ ]:
it_lasso23 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
sfs = ['bc_GrLivArea' , 'log_LotArea', 'log_MasVnrArea', 'yeo_TotalBsmtSF','yeo_GarageArea','log_LotFrontage','log_OpenPorchSF']

In [ ]:
it_lasso24 = regx2(model= Lasso(max_iter=3000),
x_train = model_vars_data[sfs],
x_test = test_vars_data[sfs],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
ridge_keep = to_keep(it1_lasso,0.000053)

In [ ]:
it_ridge1 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[ridge_keep],
x_test = test_vars_data[ridge_keep],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
ridge_keep = to_keep(it_ridge1,0.001)

In [ ]:
it_ridge2 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[ridge_keep],
x_test = test_vars_data[ridge_keep],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
ridge_keep = to_keep(it_ridge2,0.002)

In [ ]:
it_ridge3 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[ridge_keep],
x_test = test_vars_data[ridge_keep],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
ridge_keep = to_keep(it_ridge3,0.0025)

In [ ]:
it_ridge4 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[ridge_keep],
x_test = test_vars_data[ridge_keep],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
ridge_keep = to_keep(it_ridge4,0.0026)

In [ ]:
it_ridge5 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[ridge_keep],
x_test = test_vars_data[ridge_keep],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
ridge_keep = to_keep(it_ridge5,0.0027)

In [ ]:
it_ridge6 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[ridge_keep],
x_test = test_vars_data[ridge_keep],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
ridge_keep = to_keep(it1_lasso,0.000053)

# romove bc_GrLivArea

no_GrLivArea = list(filter(lambda x: x != 'bc_GrLivArea',ridge_keep))

In [ ]:
it_ridge7 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[no_GrLivArea],
x_test = test_vars_data[no_GrLivArea],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keep8 = to_keep(it_ridge7,0.0006)

In [ ]:
it_ridge8 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[keep8],
x_test = test_vars_data[keep8],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keep9 = to_keep(it_ridge8,0.0001)

In [ ]:
it_ridge9 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[keep9],
x_test = test_vars_data[keep9],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keep10 = to_keep(it_ridge9,0.001)

In [ ]:
it_ridge10 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[keep10],
x_test = test_vars_data[keep10],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keep11 = to_keep(it_ridge10,0.002)

In [ ]:
it_ridge11 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[keep11],
x_test = test_vars_data[keep11],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keep12 = to_keep(it_ridge11,0.003)

In [ ]:
it_ridge12 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[keep12],
x_test = test_vars_data[keep12],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keep13 = to_keep(it_ridge12,0.003)

In [ ]:
it_ridge13 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[keep13],
x_test = test_vars_data[keep13],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keep14 = to_keep(it_ridge13,0.004)


In [ ]:
it_ridge14 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[keep14],
x_test = test_vars_data[keep14],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keep15 = to_keep(it_ridge14,0.005)


In [ ]:
it_ridge15 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[keep15],
x_test = test_vars_data[keep15],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keep16 = to_keep(it_ridge15,0.006)


In [ ]:
it_ridge16 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[keep16],
x_test = test_vars_data[keep16],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keep17 = to_keep(it_ridge15,0.0063)

In [ ]:
it_ridge17 = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[keep17],
x_test = test_vars_data[keep17],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
# keep17 = to_keep(it_ridge15,0.0063)
keeps= list(filter(lambda x: x != 'bc_GrLivArea',model_vars_data.columns))

In [ ]:
it_ridge18a = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_ridge18a,0.0002)

In [ ]:
it_ridge18b = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]},
cv_ = 5,
linear_reg = True,
tree_based = False)

In [ ]:
keeps = to_keep(it_ridge18b,0.0005)

In [ ]:
it_ridge18c = regx2(model= Ridge(max_iter=3000),
x_train = model_vars_data[keeps],
x_test = test_vars_data[keeps],
y_train = np.log(price_train.SalePrice),
y_test = np.log(price_test.SalePrice),
params = {"alpha": [0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000, 10000]},
cv_ = 5,
linear_reg = True,
tree_based = False)